In [1]:
from orm_loader.helpers import configure_logging, bootstrap
from omop_alchemy import get_engine_name, load_environment
import sqlalchemy as sa

configure_logging()
load_environment()

engine_string = get_engine_name('cdm')
engine = sa.create_engine(engine_string, future=True, echo=False)

bootstrap(engine, create=True)

2026-01-26 21:26:57,912 | INFO     | sql_loader.omop_alchemy.config | Environment variables loaded from .env file
2026-01-26 21:26:57,912 | INFO     | sql_loader.omop_alchemy.config | Default database engine configured


In [2]:
from omop_alchemy.cdm.model.vocabulary import Concept, Concept_Relationship
from omop_alchemy.cdm.model.clinical import Condition_Occurrence
from sqlalchemy.orm import sessionmaker

In [3]:
Session = sessionmaker(bind=engine, future=True)
session = Session()

In [4]:
from omop_alchemy.cdm.model.extended.concept_resolver import OMOPConceptResolver, ConceptValidationMixin
from orm_loader.helpers import Base


In [5]:
related_concept = sa.alias(Concept, name='related_concept')

q = (
    sa.select(
        Concept.concept_id,
        Concept.standard_concept,
        Concept_Relationship.relationship_id,
        related_concept.c.concept_id.label('related_concept_id'),
        related_concept.c.standard_concept.label('related_standard_concept'),
    ).join(
        Concept_Relationship, Concept.concept_id == Concept_Relationship.concept_id_1
    ).join(
        related_concept, Concept_Relationship.concept_id_2 == related_concept.c.concept_id
    ).where(
        Concept_Relationship.relationship_id == 'Subsumes'
    )
).subquery()

In [6]:
class TestMapper(OMOPConceptResolver, ConceptValidationMixin, Base):
    __table__ = q

    concept_id = q.c.concept_id
    standard_concept = q.c.standard_concept
    relationship_id = q.c.relationship_id
    related_concept_id = q.c.related_concept_id
    related_standard_concept = q.c.related_standard_concept

In [7]:
table = TestMapper.get_queryable_table(session)
cols = TestMapper.concept_id_columns()

In [8]:
violations = TestMapper.referenced_concept_violations(session)

In [9]:
import pandas as pd

pd.DataFrame(violations['related_concept_id'], columns=['Invalid Related Concept IDs'])

,Invalid Related Concept IDs
0,37109760
1,37109761
2,37109762
3,42598409
4,3170326
...,...
3181,37109755
3182,37109756
3183,37109757
3184,37109758


In [10]:
class CoT(Condition_Occurrence, OMOPConceptResolver, ConceptValidationMixin):
    pass

In [11]:
CoT.referenced_concept_violations(session)

{'condition_type_concept_id': {32544, 32545, 42539609, 45754907}}